In [2]:
import sys 
sys.path.append('../../')
from main_conditional_lm import LLM
from main_utils import Trie

In [4]:
model = LLM.load_from_checkpoint(
    '../../models/product_title_taxonomy_classification/version_4/epoch=0-step=996156.ckpt/pytorch_model.bin', strict=False)

INFO:root:Unused kwargs when getting t5-large: {}
/opt/conda/envs/py38/lib/python3.8/site-packages/pytorch_lightning/core/saving.py:263: UserWarning: Found keys that are in the model state dict but not in the checkpoint: ['transformer.encoder.embed_tokens.weight', 'transformer.decoder.embed_tokens.weight', 'transformer.lm_head.weight']
  rank_zero_warn(


In [5]:
model.eval()
model.cuda()
tokenizer = model.tokenizer

In [11]:

outs = model.transformer.generate(
    **model.tokenizer(
        "Top-down categorize Aliexpress product: " + \
            "High-tech Dildo vibrator XL", 
        return_tensors='pt'
    ).to('cuda'), 
    num_beams=10, num_return_sequences=10, output_scores=True, return_dict_in_generate=True, length_penalty=0, max_new_tokens=50
)
model.tokenizer.batch_decode(outs.sequences), outs.sequences_scores.exp(), outs.sequences_scores.exp().sum()

(['<pad> beauty & health > sexual wellness > sex toys > anal expander</s><pad><pad><pad>',
  '<pad> beauty & health > sexual wellness > sex toys > vibrators</s><pad><pad><pad><pad><pad>',
  '<pad> beauty & health > sexual wellness > sex toys > anal plug</s><pad><pad><pad><pad>',
  '<pad> beauty & health > sexual wellness > sex toys > dildos</s><pad>',
  '<pad> beauty & health > sexual wellness > sex toys > clitoris stimulation</s><pad><pad>',
  '<pad> beauty & health > sexual wellness > sex toys > anal sex toys</s><pad>',
  '<pad> beauty & health > sexual wellness > sm products > gags & muzzles</s>',
  '<pad> beauty & health > sexual wellness > sm products > masturbation machine</s><pad><pad>',
  '<pad> beauty & health > sexual wellness > sm products > adult games</s><pad><pad><pad><pad><pad>',
  '<pad> beauty & health > sexual wellness > sm products > sex furniture</s><pad><pad><pad>'],
 tensor([0.2459, 0.2404, 0.1402, 0.0812, 0.0425, 0.0202, 0.0161, 0.0151, 0.0113,
         0.0101], 

In [4]:
import dvc.api

In [5]:
resource_url = dvc.api.get_url(
    'data/taxonomy/wish_newtax.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
)

INFO:asyncssh:Opening SSH connection to github.com, port 22
INFO:asyncssh:[conn=0] Connected to SSH server at github.com, port 22
INFO:asyncssh:[conn=0]   Local address: 192.168.0.3, port 32798
INFO:asyncssh:[conn=0]   Peer address: 140.82.114.3, port 22
INFO:asyncssh:[conn=0] Beginning auth for user git
INFO:asyncssh:[conn=0] Auth for user git succeeded
INFO:asyncssh:[conn=0, chan=0] Requesting new SSH session
INFO:asyncssh:[conn=0, chan=0]   Command: git-upload-pack 'junwang-wish/query_understanding_data.git'
INFO:asyncssh:[conn=0, chan=0] Received exit status 0
INFO:asyncssh:[conn=0, chan=0] Received channel close
INFO:asyncssh:[conn=0, chan=0] Channel closed
INFO:asyncssh:[conn=0] Closing connection
INFO:asyncssh:[conn=0] Sending disconnect: Disconnected by application (11)
INFO:asyncssh:[conn=0] Connection closed
INFO:asyncssh:Opening SSH connection to github.com, port 22
INFO:asyncssh:[conn=1] Connected to SSH server at github.com, port 22
INFO:asyncssh:[conn=1]   Local address: 

In [6]:
import pandas as pd

In [7]:
df_tax = pd.read_json(resource_url, lines=True)

INFO:aiobotocore.credentials:Found credentials in environment variables.


In [8]:
df_tax.head(2)

,category_tree_version,id,category_path,is_leaf
0,v1.2.1,4658,Security & Protection > Building Automation,True
1,v1.2.1,4659,Security & Protection > Door Intercom,False


In [9]:
paths = df_tax[df_tax['is_leaf']]['category_path'].apply(lambda x: ''.join(['[' + i.strip().lower() + ']' for i in x.split(' > ')])).tolist()

In [10]:
len(paths)

5291

In [11]:
trie = Trie([
    [tokenizer.pad_token_id] + tokenizer.encode(i) + [tokenizer.eos_token_id] for i in paths
])

In [12]:
def constraint(batch_id, sent):
    return trie.get(sent.tolist())

In [19]:
outs = model.transformer.generate(
    **model.tokenizer(
        "Categorize Aliexpress product: " + \
            "ALONG FIT High Waisted Tummy Control Leggings-Yoga-Pants with Pockets Leggings for Women Workout Squat Proof Tights", 
        return_tensors='pt'
    ).to('cuda'), 
    num_beams=5, num_return_sequences=5, output_scores=True, return_dict_in_generate=True, length_penalty=0, max_new_tokens=50,
    prefix_allowed_tokens_fn=constraint
)
model.tokenizer.batch_decode(outs.sequences), outs.sequences_scores.exp(), outs.sequences_scores.exp().sum()

(["<pad> [women's clothing][bottoms][fashion leggings]</s><pad><pad><pad><pad><pad><pad>",
  '<pad> [sports][fitness & body building][yoga][yoga pants]</s><pad><pad><pad>',
  '<pad> [mother & kids][pregnancy & maternity][leggings]</s><pad><pad><pad><pad>',
  '<pad> [sports][sports clothing][sets/suits][trainning & exercise sets]</s>',
  '<pad> [sports][sports clothing][tights][running tights]</s><pad><pad><pad><pad><pad>'],
 tensor([0.6398, 0.0250, 0.0215, 0.0206, 0.0196], device='cuda:0'),
 tensor(0.7264, device='cuda:0'))

In [13]:
df = pd.read_json(
    dvc.api.get_url(
        'data/wish_products/wish-mturk-labelled-09202022-clean-joinedlance.json',
        repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
    ), lines=True
)

INFO:asyncssh:Opening SSH connection to github.com, port 22
INFO:asyncssh:[conn=3] Connected to SSH server at github.com, port 22
INFO:asyncssh:[conn=3]   Local address: 192.168.0.3, port 32828
INFO:asyncssh:[conn=3]   Peer address: 140.82.114.3, port 22
INFO:asyncssh:[conn=3] Beginning auth for user git
INFO:asyncssh:[conn=3] Auth for user git succeeded
INFO:asyncssh:[conn=3, chan=0] Requesting new SSH session
INFO:asyncssh:[conn=3, chan=0]   Command: git-upload-pack '/junwang-wish/query_understanding_data.git'
INFO:asyncssh:[conn=3] Closing connection
INFO:asyncssh:[conn=3, chan=0] Closing channel
INFO:asyncssh:[conn=3] Sending disconnect: Disconnected by application (11)
INFO:asyncssh:[conn=3] Connection closed
INFO:asyncssh:[conn=3, chan=0] Closing channel due to connection close
INFO:asyncssh:[conn=3, chan=0] Channel closed
INFO:asyncssh:Opening SSH connection to github.com, port 22
INFO:asyncssh:[conn=4] Connected to SSH server at github.com, port 22
INFO:asyncssh:[conn=4]   Loca

In [14]:
df.head(2)

,pid,title,category,text,lance_predicted_category_id,lance_predicted_category,lance_predicted_is_leaf
0,5ce7ad18c04b4c486820a407,Fantasy Master - 681 - sv√§rd,"[education & office supplies, cutting supplies...",Fantasy Master - 681 - sv√§rd -> [education & ...,2705,"[home & garden, home decor, ornaments, swords]",True
1,610dee5b63838066f029717e,Silent Red Thing Silent Hill Pyramid Head Horr...,"[home & garden, home textile, bedding, blankets]",Silent Red Thing Silent Hill Pyramid Head Horr...,2784,"[home & garden, home textile, bedding, throws]",True


In [15]:
df['category_text'] = df['category'].apply(lambda x: ''.join(['[' + i + ']' for i in x]))
assert df['category_text'].apply(lambda x: x in paths).all()

In [30]:
from tqdm import tqdm

In [40]:
recs = []
for rec in tqdm(df.to_dict('records')):
    outs = model.transformer.generate(
        **model.tokenizer(
            "Categorize Aliexpress product: " + \
                rec['title'], 
            return_tensors='pt'
        ).to('cuda'), 
        num_beams=2, num_return_sequences=2, output_scores=True, return_dict_in_generate=True, length_penalty=0, max_new_tokens=50,
        prefix_allowed_tokens_fn=constraint
    )
    preds = model.tokenizer.batch_decode(outs.sequences, skip_special_tokens=True)
    probs = outs.sequences_scores.exp()
    rec['t5_predicted_categories'] = [pred[1:-1].split('][') for pred in preds]
    rec['t5_predicted_category_prob'] = probs.detach().cpu().tolist()
    recs.append(rec)

100%|██████████| 6401/6401 [45:56<00:00,  2.32it/s]


In [41]:
df_pred = pd.DataFrame(recs)


In [42]:
df_pred.head(2)

,pid,title,category,text,lance_predicted_category_id,lance_predicted_category,lance_predicted_is_leaf,category_text,t5_predicted_categories,t5_predicted_category_prob
0,5ce7ad18c04b4c486820a407,Fantasy Master - 681 - sv√§rd,"[education & office supplies, cutting supplies...",Fantasy Master - 681 - sv√§rd -> [education & ...,2705,"[home & garden, home decor, ornaments, swords]",True,[education & office supplies][cutting supplies...,"[[home improvement, home appliances, home appl...","[0.01616569049656391, 0.01278897374868393]"
1,610dee5b63838066f029717e,Silent Red Thing Silent Hill Pyramid Head Horr...,"[home & garden, home textile, bedding, blankets]",Silent Red Thing Silent Hill Pyramid Head Horr...,2784,"[home & garden, home textile, bedding, throws]",True,[home & garden][home textile][bedding][blankets],"[[home & garden, home textile, bedding, blanke...","[0.35011327266693115, 0.1273525208234787]"


In [44]:
for depth in range(7):
    tmp = df_pred[df_pred.category.apply(len) > depth]
    if len(tmp) > 0:
        print('Depth:', depth)
        print('Size: ', len(tmp))
        print('# Unique Targets', len(set(tmp.category.apply(lambda x: ''.join(x[:depth+1])))))
        print('ACC T5: ', (tmp.category.apply(lambda x: ''.join(x[:depth+1])) == \
            tmp.t5_predicted_categories.apply(lambda x: ''.join(x[0][:depth+1]) if len(x[0]) > depth else None)).mean())
        print('ACC Lance: ', (tmp.category.apply(lambda x: ''.join(x[:depth+1])) == \
            tmp.lance_predicted_category.apply(lambda x: ''.join(x[:depth+1]) if len(x) > depth else None)).mean())
        print('-' * 100)

Depth: 0
Size:  6401
# Unique Targets 25
ACC T5:  0.6777066083424464
ACC Lance:  0.8042493360412436
----------------------------------------------------------------------------------------------------
Depth: 1
Size:  6401
# Unique Targets 203
ACC T5:  0.5192938603343228
ACC Lance:  0.6231838775191376
----------------------------------------------------------------------------------------------------
Depth: 2
Size:  6401
# Unique Targets 868
ACC T5:  0.37478518981409153
ACC Lance:  0.5063271363849399
----------------------------------------------------------------------------------------------------
Depth: 3
Size:  2475
# Unique Targets 869
ACC T5:  0.2404040404040404
ACC Lance:  0.4
----------------------------------------------------------------------------------------------------
Depth: 4
Size:  27
# Unique Targets 18
ACC T5:  0.2222222222222222
ACC Lance:  0.4074074074074074
----------------------------------------------------------------------------------------------------
